<a href="https://colab.research.google.com/github/GPaz027/Cariety-ML/blob/develop/models/yolo/YOLO_NAS_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO NAS CARIETY - Utilizacion
## Carga y utilizacion del modelo ya entrenado

## Montar Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalar super-gradients - Suele tener problemas en local

In [2]:
# One time installation.
!pip install super-gradients

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... 

## Importar librerias necesarias

In [1]:
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train,
    coco_detection_yolo_format_val
)
from super_gradients.training import models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import (
    DetectionMetrics_050,
    DetectionMetrics_050_095
)
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from tqdm.auto import tqdm


The console stream is logged into /root/sg_logs/console.log


[2023-08-15 19:51:03] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-08-15 19:51:07] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-08-15 19:51:13] INFO - utils.py - NumExpr defaulting to 2 threads.
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-08-15 19:51:15] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-08-15 19:51:15] WARNING - export.py - Failed to import pytorch_quantization
[2023-08-15 19:51:15] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

import cv2
import requests
import zipfile
import glob
import random
import torch

## Definicion de Variables de entorno

In [3]:
ROOT_DIR = '/content/drive/MyDrive/yolo_data'

SAVE_PATH = '/saved_images_temp'

In [4]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

## Carga del Modelo

In [5]:
model = models.get(
    model_name='yolo_nas_s',
    checkpoint_path=ROOT_DIR + '/checkpoints/yolo_nas_s/ckpt_best.pth',
    num_classes = 1
).to(device)

[2023-08-15 19:51:20] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/drive/MyDrive/yolo_data/checkpoints/yolo_nas_s/ckpt_best.pth EMA checkpoint.


# Fast API

## Instalación

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

## Importar lo necesario de Fast API

In [34]:
from fastapi import FastAPI
import uvicorn
import nest_asyncio
from pyngrok import ngrok

from pydantic import BaseModel
import base64
import PIL
from io import BytesIO

## Implementacion Fast API

In [35]:
def save_image(base_64_image, relative_save_path):
  image_path = ROOT_DIR + relative_save_path + "/Resultado.jpg"
  im = PIL.Image.open(BytesIO(base64.b64decode(base_64_image)))
  im.save(ROOT_DIR + relative_save_path + "/Resultado.jpg")
  return image_path


In [36]:
app = FastAPI()

class Image(BaseModel):
    base_64: str

@app.get("/")
async def root():
    return {"message": "Hello World"}

@app.post("/image")
async def upload_image(image: Image):
  base_64 = image.base_64
  print(image)
  path = save_image(base_64, SAVE_PATH)
  print(path)
  out = model.predict(path)

  return { "base_64": base_64} #, "yolo_output": out }



## Ejecutar y Exponer la Aplicacion - Ngrok
### Ver URLs en los Logs

In [37]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

[2023-08-15 20:10:58] INFO - ngrok.py - Opening tunnel named: http-8000-838e08ed-12e7-484c-82bb-2c6c3cd8205b
[2023-08-15 20:10:58] INFO - process.py - t=2023-08-15T20:10:58+0000 lvl=info msg="no configuration paths supplied"
[2023-08-15 20:10:58] WARNING - process.py - t=2023-08-15T20:10:58+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml
[2023-08-15 20:10:58] INFO - process.py - t=2023-08-15T20:10:58+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
[2023-08-15 20:10:58] INFO - process.py - t=2023-08-15T20:10:58+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
[2023-08-15 20:10:58] INFO - process.py - t=2023-08-15T20:10:58+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
[2023-08-15 20:10:58] INFO - process.py - t=2023-08-15T20:10:58+0000 lvl=info msg="client session establis

Public URL: https://043b-34-83-162-49.ngrok.io


[2023-08-15 20:11:07] INFO - process.py - t=2023-08-15T20:11:07+0000 lvl=info msg="join connections" obj=join id=e6fe3963f5f0 l=127.0.0.1:8000 r=168.226.85.113:53189


INFO:     168.226.85.113:0 - "GET / HTTP/1.1" 200 OK
INFO:     168.226.85.113:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


[2023-08-15 20:11:40] INFO - process.py - t=2023-08-15T20:11:40+0000 lvl=info msg="join connections" obj=join id=2c4470de30bb l=127.0.0.1:8000 r=168.226.85.113:53194
[2023-08-15 20:11:40] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


base_64=',/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCAEAAQADASIAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAAAwQCBQEGBwAI/8QAQxAAAQMDAgQEAwcCBAMHBQAAAQIDEQAEIRIxBRNBUQYiYXGBkaEHFCMyscHwQtEkM+HxFRZSQ0RFU2Nyk2JzgpKy/8QAGwEAAgMBAQEAAAAAAAAAAAAAAgMAAQQFBgf/xAAoEQACAgEEAQQCAgMAAAAAAAAAAQIRAwQSITFRBRMiQSNxFIFCYaH/2gAMAwEAAhEDEQA/ANzdBJB6dAetQ1EkSOnSmHZLR6QaBqkbGAa88dMGZmBEbmTBoaSTg5BxBo6gnODB7nascsNqBTIzg9qhACtWraBnJrLpg9JBiprMggEbTWNRIOSDtAH1qrILlSj6RUElSoC8Qd6IsJIxkx1/ahO6ggFCQZPn9P5tVNhII6+Ut7A9QIFBW+qToUJG6MVjXKPMjM4EzUEpEyEkfGgssgtx4LV+JPb2rKL3S5Dw8hEE+lYeIQj8xyYGCRQnQgfmUAhHcipyWHeWySCh0efAxH+1KuzywvmCJjFBUdROBE/0maBqCyAEnQZ2q7YVB0nYoVIOR8KHqMflGCR8KG6SECB8KXcc61dl0Mrc8hIGN80HfA3oS3Z06ZgnIqCbgJ8vwJqyUFW5MZAApa8fSkEqPTHrQXbo5IAPT3pHit2lu1ddWICEkn0EUUVbJuopeBE3XiZ51adEvaD12x+gruDQi2bDIkojTnY+tcS+zdJc4ozzJJMrJxXbGXgXIzAEx0FdXp

[2023-08-15 20:11:52] INFO - process.py - t=2023-08-15T20:11:52+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
[2023-08-15 20:11:52] INFO - process.py - t=2023-08-15T20:11:52+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


## Prueba del Save y del Path - Test de que funciona

In [11]:
BASE_64 = ",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCj/wAARCAEAAQADASIAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAAAwQCBQEGBwAI/8QAQxAAAQMDAgQEAwcCBAMHBQAAAQIDEQAEIRIxBRNBUQYiYXGBkaEHFCMyscHwQtEkM+HxFRZSQ0RFU2Nyk2JzgpKy/8QAGwEAAgMBAQEAAAAAAAAAAAAAAgMAAQQFBgf/xAAoEQACAgEEAQQCAgMAAAAAAAAAAQIRAwQSITFRBRMiQSNxFIFCYaH/2gAMAwEAAhEDEQA/ANzdBJB6dAetQ1EkSOnSmHZLR6QaBqkbGAa88dMGZmBEbmTBoaSTg5BxBo6gnODB7nascsNqBTIzg9qhACtWraBnJrLpg9JBiprMggEbTWNRIOSDtAH1qrILlSj6RUElSoC8Qd6IsJIxkx1/ahO6ggFCQZPn9P5tVNhII6+Ut7A9QIFBW+qToUJG6MVjXKPMjM4EzUEpEyEkfGgssgtx4LV+JPb2rKL3S5Dw8hEE+lYeIQj8xyYGCRQnQgfmUAhHcipyWHeWySCh0efAxH+1KuzywvmCJjFBUdROBE/0maBqCyAEnQZ2q7YVB0nYoVIOR8KHqMflGCR8KG6SECB8KXcc61dl0Mrc8hIGN80HfA3oS3Z06ZgnIqCbgJ8vwJqyUFW5MZAApa8fSkEqPTHrQXbo5IAPT3pHit2lu1ddWICEkn0EUUVbJuopeBE3XiZ51adEvaD12x+gruDQi2bDIkojTnY+tcS+zdJc4ozzJJMrJxXbGXgXIzAEx0FdXpUc3t2N2zqm2QysAo1EEpJkj+RTSUlTrHLJIbUZ83SP7wKr2VJDTaUZQAI80xjG9NNuktkNkIWsCJE9unt+1CmXRaIZAuluZgpCNM4xJx23M+wolw4W24kgkwMSKiDMHb2qS0lQMmR2pyADWjyeSjXGvsDTzKtVU7SUu3Xk0FaEwe6Ov9vpVgy4AgFGMYFHBgNFi37T8a8fKDo3qDboAgnPpUkEqJxTAAiJ0aic0VIOnfeoa5r2VJ281WUaFA5EkCAfrQFpEQUyDvTDqQGdS8GdzQ0f5gE+cj9K88dMCEnlyAd+1RIJgkYHymmoEIk9em9DQCQMFB9hVUWKFOlJHrM+lQJ/PESMGRTa0z0PTeoFsKWBOPapRBZQBEmRHel3R/lwdyZAHpvT6wc4770o60kOFSPzrzI+OKqQSBr/AD7fSgQCsx1ztRl5aA7neg5QDtvuetLYSMEGCE9T0pZWFyIwI96M8RpGuI7TSSnDrJkwsbelUEgjgH9VASuFhSRBntWbt+BED2oCXSSPSjRdE1wEHcEDvVctQccIGED0mjXbkJXByarVXYaJHX6UZaQws8tOnVIPUiknHlJ7QOlYduC4IgwKrbm4IhKI19MzURdDbj8fkSMDvVD4jfX/AMMfMRrAQRPcgfvViC8QgIEGPM4o71qvi2+Q5dMWaFJcU2dbhSdvT9afhVyQrLxFm2/ZmyQrmHtE11BkhOZImDj5Zj3rRPs8aDdjzYya3ppxIrRN8mVIa8wccCCCSJgnHxp20S2U8zJJ8+5IPt9KqLdpxd25cFyWikIQmNu/zx8qt2CJIEgxNDB8ltF3bOBxtBmR0pk6oARvtvSVuowE9RtToOIjatafBnZBieauRBn6TTSDgScigoIOhW4PWiKT1JMT0okCxpEBvUSd9qfay3piKQZVK4qwSkqEqpqFsk2IGvvtRNxNQ1SuEiU1NJACug7UaKOfuz+IBuRie9e5aEgTpWF9/wCfCgugh7Ch5BPtnaspfadB0ONrEGROK85uOpQRxslwhGB3HevQIKQTr/SsF9IiCJBxnegBST+Q/XPxqbiUeKlAyQCiJwY+nzrHmjCsycV7mwT5gD/7qCt4OSErAEyYVv2qbkFR5R1DGAMfExQndwpe7cxHqIqYhoISwlDbbYgJCYAHYdqB94TKwSFrHSaFstIA8U6J30CYE/zrQ0wSda4RMxFZVetoM6RHzpZd2zyy44dAH9c0Fh0Qu1FGAAZ7H50kt7Ss4iKjccUtHI0ElA7iJzSb3ELboVjuAd6tIJII+8CTpyQOlIvXZbUYPk9aE7fW4WZJ37/6VXvcRs+bOnHQaqYkENrvHXQdAGgdRQ3nDAEQYnFKq4vbiYTHoD/pSy+LsTOkxjdVMogUuuPulLJxP5u1eW2m1RGnmOd6QuONKbww3HbFVjnELt7+rHpipsbKsvFl11J322rTuM288baX5PMIIA6jv8xVwq6fU2AsNn3BpRpo3XHWGoR5AMI2BP8ABTsHDE5eUdQ8Ks/duFW4I3E+9bAg60Kj4VV2aQ2yhsCABFWVqYAk1JvkGMQlheNOp0zyzqLQGoEmBPzjP8iry3BKgsFZGxBpJhAC0yJ/arNlYFFjAkWFqmB56sEBQ/MZG8VWWytTmnON561YqE6DJ8hkitcDJIMidAIojc5BGMRQEDSDjKzknrTCPLOCacgGFtyG14GTVo3+X1qrbgO9celPLcDbHqdqOADCs4BAHxoV4yXANBWJOYphACUADas5JMjHejBOO/cX23y4ErJqLto8CPKudgYq8N23GUrJO+K8m6bBASkkk5rzO1eTs/LwUKbO5SFytzfAVuPShrtn0kggyT8/5+1X/wB8bIlLZB76dhUTdok6Ur69KravIVy8GvuWdyTzFlcjYkTS33O8K3Fc5xxBJwobZ2kHbE1sf3xuFkMrJO2Kwi+3C2VgEDbOZ64qUifLwa4tq6Gg/izEb9Nv2que4dfBRCGrh0LnY5Wd+p9t620vQClKVx6g9/pUF3jgdCU27sTlUCD7fWokXb8GoXDN3a+UhxA6Ej+1KoYefZOtxawOkzNbjctffEAuMLQuTIJnFK8ltkGWVk91ij2IJN+DSHbd4Ewooz1O1KuWrxBC3AZHWt8dDCjJaJHeAafRwjgztsj7yJcIk6TET0oo476ZHOu0csVbvEkahjpUDw9ck6kE9c101HAOCPInkPoz/W4BWVeGODK/KCB6vU1Yn5BeReDlosniYDhGeiqInhj2jLs5710p7wtwXOB/83+tL/8AKliD+AyFrkQecduver9qRW9HPWeGtCee7HaKmtu2tWi5oJjua3x3wyyXgGbZt0nBJdXPyFSHg5l5wMvnloXM4J9utWsTsvejnC1FSGiloIdXBISPkKc8McKUL5d3cmMlZnpW/ueDrRm6FwoStCpACY9vrmi2fD0tuLMRTVD2wLUwbLZLaFKBBIHkPSnGkCIV+Xsaw4NOIiNhUGlplIHUYE1nbD2lq0oJid6dYUVNBSoGKqbd0OIE7/PY1ZWmpSyCBoxH702DM8y1tM5Gxq2tlE4I9zVWz67VY2jgitmMxyG1AkYrKDt614GQKwoE7d+tOFBrZ1LhMYgkZprSHXGyoedB3NKNiI8sZmnLUk3AMeTTM+tMRTHRIMRWFE9qkleek1hR8u1MAOFo4yoSVmRExTltxAOuLCj0rU0GRg7p2ptsqheT+WvOUj0+wsrvilwm+0Ie0IkHYVcLdI0id60LiSiL1B1H8o61s7zqhyCCelNyQSSYiCdstlrU2klNUdtxa6ev+SdAQZ2FO3dyoNLA8xitY4Vdj/jISRnmKFMwwi0+BeXcmjZbx51KOXqyRiBtSj19fBsgPfiAYJ70xfzypSc96r3tRbncxv3oFFBlbw/iHFX3Vi6eKwjCiEgAH0o1hxDin/N/D7BV0tdlcTzELgz8SJFLMXpbaDr5aAiYSqapOK8RDzxcwCMD0FMtJ9ArFLJ0bx4r4rwi0bLTbyC+Jw15/wDStOR4sfYb5baUKE+XmDI+Va6twumTt70MBOsRkUDaZpx4aXJd3HinibyY55Qn/wBMaP0pNXF+JK3vrr/5l/3rFvyzA8vtVqxZ27oGsUl5FAdsSKdHFL9JJF9cgnch5Qpyz8VcatBCL91xEyUPHmT6Scx7U29wlsnyA1XvWKm1wU0UM8X0DUGbnwL7TzbkN8U4a3y9WXLTyECI/Idz6yK3zhXFOG8dCHuGXDb+hUqjBRnqDkda4YqxBrNsy7Z3CHrV1bD6NnG1FBHxrTDIZ8mmi+UfQN63GAKp1JS2gyY9a1bgXjtxxoW/G/OdhcoEH4gfqPlWzgtPthxl0ONryCCCI9KmSd9GdY3DhlfcJOsHrEe1ASMo1HIzPembqOfHr3pReHQewP7VlfY76HrdYHmAGvAJ61a2jyVqjaqJCgqRPpT1tIjGRgKpsGZ8iNjZcEgU6y5+IANzVOy4oLQoKGDkRuKs7AgpCjlYJE1tgzFND33iLpDGPyzM7UwhUrHcCg8hCnUPH86cYoiAZmK0KxDGkGXAmMd6ZQ2pJQlsHlrMGP6cUBmB1kk1YW+B/M0aBI2ii4SI/IIPvR1STBobOltxcAAHJ9amsyvAoyj5tt4Bb1DMGrFgEgYyUxQWGCS1iZJFWjLPKebSYiuDsPTOZrXF2j97YJEnTWw3K0i3YM5ABqv8ToSzctkkAEECmENardCiN0oM1pyQ+EREJrcyaXg6CCCCcVrlm2pPiJB/9UiK2dpmHPTVj5VRpSU+Ih1HOH6UWnh3+heZ9GyLblrSQYNVPEX2rS2BOREJHer51vyGRAMyZrn/ABR8uveT/LGGx6Vn6NGLH7jK69uMwgBA7AQBVS85K+tPXaDBJwTSCWyo7Y96o17NoFLnr5O1eUvoDj2rGnoKwlJqBJDNtIIJJnpV1Z3IDgBiqJAPypm3XCwZpWSFltKjdbd1tbYAI2oF8yFIkb1V2L5nEf3p8uqUnvWNYnF8GNxaZWLGcUJyfemLlJSZAxSrnea34w0LqkKmrTgnHbnhjo0HU0T5mycH+xqtJ3oKtO9Potqzptrxa24ly3rdzQsJPMZO4n9dvrRXSEpmOlctRcOMrCmzBBkEVf8ADvEzkcu7JcR3H5x/elSxvtCHCjb2nZKjNPs3MpQEQSD3qhYfZcaDrLoKD1pi2ukha1QCsYFSBnyI3S3Vg6JgelW9gRriAMTArVuHOFxRcDmFiAO1bGwo6goCSR3rZjMGQugYHtWUnUMTnegW7hcC5GAYptlJJrWjOw9s22QQCCQYx0xtVk0NI2pa2TH5R9KYeWQlPLST6d6YkATCZnV7V7bepIHnrDqJG3vVkOMWVm0EQCD1p9NqzzNRIkVQ8MJFxBnNOlARcb/Wsb01HUWosr/HVot1lhVu2VkHpRLRp02LUiF8sCDVuTLCB2qaQCNqHI/io+A4cPd5KwMK1521TVA8nT4iHq6g/StyjpWncSdSjxOhqDrKkLq9NG2/0VmnwjYeOoWnh7uiJX5PmYP0rR+Is/d/MR7Ct14oVFspJMc0foa1vjDGsOkmelYpqmdHTPg1aOY4vGsrwKUfbSCrRt3pxUhXkMHvQFN9NHxoGzfsK8tn+kYrOkTThZkJJioLbxECqsFoWQPlRUzj0qQGc1BZjIFF2ANW7vLzNNtXvlycVTqcOiFVhK5VHX3oHAHZZsfMS4N6VuEwDpNKWju2RTqxqQCOtFBUA1tEo8lLKEfCnFo0ahFLrAiZp6KFXTigcwoMzTCxvS605piQE0P2HE3LUyFGDuO9XlpxVTrqCjKF960096Pw+5Uw9AOJoXj+zHl4O1cGdhKOxM+1btYtxg1zfwtdh5pudj3rpNj5kgxvtTcSOdlLJkpgAHrTtuCDnGdqUZb0gx1OMzT9uJIrWjMx+3EiRRFGYB71BlQSYT1NEUkcwGc9aYCEmNztU1EGKgmFmszmrAPn5mQ4FeuIq4fQToVEzVojwrxUoH+ENWTXhfiRYALUEVeRI0QyUa8gbiKO0kkVfW/hXiIc8zeNqca8MXacEViyY+TVDMqNZ5Z61oXiJpxHjC2VAg6I+ddp/wCV7gnNRX4NK3NTjaCvoTV4X7bsDLkUlRzzi8WrXOfP4ZdG3sao+Kt/4VZHX1rofjrwo+34U4g8w2HHWkh2J6AgqP8A+oNcyDwc4fbh1X4cGSMkxt/b4VjzqjtaGskNy8mvuMKJ2qXKjB3p4N6sp8lZCRMEgeprPts6rKstFCdqQIe5pkQgbYq6WqFwRtQHmwrKaqtotoqikxM0JdMPjSvHTpSqzKFRVoW0BccAVE1BBEZG9QWIXWNcSflTKIMIcIOMCrm1cJQmaoULnpVrZPSmKlC5jbqZJNKuI00ZTsioKzTELFXE7Uo4nyq/SrFYM+1LLEnemIpiSx8KEpH/AEjamljtQog5NMRmyI3n7PLsOoQnsa7Xw9IcbaUZlBlMGMwRnvvXz74FcLfFCgD85n619DcKSBboUdwAKLGuTk5uGWrSSUSPlT9ukhEjOPrSluBo2kdqskNgInbE1rSMrCswXYBGPpWbtpDwW0uYWCMVC0b5brmcz8qPctcxCDMEHer+gfsggfkNTTJE9qwiI9KmlWUhP5ahZzZH2uXnLzwllZ/+8R+1NI+11H/eeFLSezbs1yhRg9RUAqTXklrc3k+gP0XSv/H/AKzsbP2tcMP+Zw2+A7+Q/vT6ftO4EqCv7w11y0T+lcP1ZkfKvKPWiXqGUW/QdM/K/s71b/aF4ad/78UH/wCppY/amEeM/Drv/jFqD6uRXzrdXTTDJdeUhCBupRgCtavvFHDGUEofLhn/ALMT/pWnFqsuTqNmPN6Lpsfc6/dH1s/xbgt/bOM/fbK4YdSUKbLgKVA4II6ivnfjNgrhl1eWRUHOS+psO9XUEYXuRkEH41z5fi99zFjaJMmElQ1T8B/ei8Hc44/xJt675gYEktkBsR7Vr+c181QjTYsenl+KTl/RuDY1YrBakkSTVzwjhrnE7XmsJJDQ/EjpU3rJTAWnlEdNqpY3Vm+c0pUay41APU0uUGQKu12ZadB0mT3oT7AMmKzzdMq0a/et5JXnFVDiTnRma2S8YV8KqnmfOAnHqauLJRTq1dd6Hk087gQaWKTB8tMTFtAhrimbZ3S4ATvQUp8u+egqbbZmRn4UZTRaqMqj60ZAoTAwDTXLJFGhdA9Mo3FLrSnqIVT2kzHWgOtyv0owKK8oHbNCUzLgDiuWiYKwJirHkyUzWfuvOwKYmKyRbQ/9n9qp3irGOoE19D2DKi0hKlECM9K5b9nfCgzL2nY7xXYuGtxoQekYp2NHD1L+VDjLOEaBgUzgKGeu1FaTobP9XpSFy7OsNglY6Doae+DMuR0eR5zbMH9qIpcggGDVfbvAgk7bJiipdXB0yCalkodyRkQKgpZ/pMxS7TbqxpWZAG9HLAA3I9qlko+aUO84kRArKUiIzUFJjIryczFeHPrBTeNHeIcJVbu2LgDDg2KQTsDM+xGMGtWc4t4ju1aG3SnsG0iT+9dBuGg822h8a0NqKwk7T699hWGWmmx+GlCE9gK3Q1OOEV8U2ch6HUzyO8jSNCT4U4pfqDt/cqyJ/FUVmn7PwxaWx/GHNIM5rc4G9KXCYkgY3q3rsk+FwNh6Xhi7at/7Fbdhq1RDLaEA5IFF5qxIY1ysRQCpZEip27hS6g9JiarFOW9NmmeOKg4pHRvspfYZ447ZXbcMPAoTJwD0k10rivhFpaisJ0me0yK4dwW9VacSQ5/0KBPf1r6c8P3Q4nwC3eDmpRQPN6139JNTjtf0eR9WhPDNZV9nGuL+Gj950gmRiCKo+MeHHrC5Da5IKdeBXafFLbDaG+by+YG5KwMk/wCta4t9m4JN0J0N+WdqLLpoyEYNXPs5Uvg6nZ5ba1gelUF9wgtqUFpg/pXXGHmWXHRKIORWi+KCXL1wjCPSsU8SSs34tRKcqNCvOH4Ggyfakl2SxWxryf3oLzH4ZV1NKo2Gt/dADnNO2tqjJIjGKadTE96xgDfNWiSRC3aEwnvTjTJJAr1s2CgU+hMYGKckLYottKemaXU3k+tWC2yTtQi2JOKNACPJk9sVbcJsi66BBoCGC4ZjFdL8C+Glvsi4W35Jx707HG2ZdVlWONl14Q4Z92t0CMxPx/n6VudpbFrzYB98+1e4fYpYbgDpVg0iYKfy9fWtaR5yUnJ2QU3obJABImKrbtp9xAa1FJKsqSMn+RViTCiARUV+cAYSuetRqykKW7aUr8wI17dyasmWmwgQnPevaRqGoA561PVy4FRIpuwaE+f+qB2qSZQI3968pyFeUb1lSwT5dWDVkPl1bhk7kVJC4G01lSQT7elYIBxvFeIpH1lMzzQEjBoYcnMHeslIG4r2I2xUpFHubHSordBGxrD24ztvUCAroMUaSL5AqCgdiO9SSgwMYoSlkHSmSPSptDMTFNVIXTHrUgu5GDXXvs78TK4Z+Bcal25gED+k1yK2ICAFDY1sFhcBGhSZMV1NPm2uzk+oaZZo7Wdg8VOuXGm5xy4nBrSLy8S4y5pdKQsRipq46XrIMuSUAROMVrvEinOhUoJrfky2rRxNPpnDiRhNwEnzuGZ71jiy7NYBWSVneKoblxLbi/zntB2pS4unCjfzgYrJvOh/G5sci0LhyfjSfEXADjYDApBl10iSob7RXrlRdVvrPtS2hntULLJUTtFQQnUsJAzRCgxj6UzZ2ykSoiMbUUESfCJW7RSjf3FMoOPWpsslzpVnZcP5ixIBp6RmlNCDbSnPamWbIOOAGD6CtltuCulpbiGSW241FAwie/atqsPDLNotDqW2blyA4AFEa0T01R/MdZDYQMWXVRgUPhvwybpxt5aQLfUAVEYH9/5NdN4dZqtbFtpgI8mTj85/n+1Rs2A4AogsLy1CVSB2222FPtLAab1gIKwMAmPrWrGqOPqMryvkkqCISTnMjpREEED16jtSl9bKUUOslflIISCQCcjMeh2OJHtRmW18sKMhZRkEzBo7M9GVNCV6B8KnoPLhQHwqawQgYyTn0qSNiT86hR5I8gHzqKENtoDcYAAAJkxULx4tWbjjKQVgYBwDRWgQ2gOfn6x3qEALw4YEwJrwdGhBiJo+nSqIHpQkgZxIGx71RD5jdJ6VEkJCs1NZ9MV4iRIzXij6vESdunSYbSN8Cd6cSnHm+te5Y1zpA9anEZqFiVymSNNeSga/NRnknFAMpcntRooGlBCjpA96m0RqlSc9qn+YZqCJSshU71a5JuCgEZmIp+zuOUsAglB39KRVHw61AvwZG1aMM9vAjKkzcbZJdbGggj3rF3bKjUlIxvNV/hziDTxDTitC/XatsNk7omMVr3nFz/jlyaY/aFedMTSLtosSdM1vDtrIkpPwpY2jDmHPmKimRahI0F5tST/lxNDQ24okIHyroDvCLRSNZ29TQl8HaYHkSKdBX2R6mJqNtYqjUsZ7Va29qAI01aJsiVQkGfSrnhXBHH3kBQAk5Jp6SRjy5/tlDZ8OW84Ahsx610DgnhYNtBziCT3CRv7Ht+tXPC+GMWQQplMrEgnHWrFxBRa3HJ18zSVpyTmP9q0wx+TkZ9U5cREEtltIbKOUEIDZ5IGgCRJAVmMySSfyEdcjs3LmwuwH4dbcKAzqJccwv8Q7CdxsPoKcdZed5baHuWAkl1IGVxo6/T4+gol+hl21aKGy4tLk6g0FlsiZ3HuMZzR0ZNwW+Bct1l53lsL8hgSYIj5ycb9KZs0NclsMpQG4JGIzO9VXEFtPcLu2W3Gi7bJGoOCCCBuQAI2kGBPyo/AU8m0Q0ptYOkLEgYGSBg9hPxok+QGuC6SPJEYoawFnSRJmfapKkYScziayktrbDqVAIyskqxEb0wWQZnlQ9JOdqlboIaQj4EUVLaHRpI9VIPQ+tePkX2AqFHlthREnKMgd68BM6xEd6864rmhJBiMRWWVcz2BzUIYUDEjavKOhtZMRHamEphEECe9Unii6+58JdIOguQ2k+px/r8Kj4VkXLo+ciSTjashPUqk+9TQgK6TU46J+VeJPq4FeBNRg9frRXUzGnesJ+UVKBsCUqmIj2oZbOaaiagoZijBFEiMwN4rD2M7UQiD9KXeJ1iKNKy2yYkjvQ177V5CjMRisrk/lo0qFyaYu5LZBBg1sXAfHV7w5AZvmxeW46EwsexqguAeWrpVaogzpGadHkHLhhkjUkdks/FvhviTYK7j7s4RlD3kj47UVVxwaC4i/tljf/MBrhywZ9ugpG6nlxT8ceTjZ9HHHzFs7kvi9k5ci3trhlxcSA2QT8hVhbWqrg+QjPeuEeDn1W3iK0XsCrR88fvXf+CKJUO9a0qZycuRw6LDhfCmg8sPtyEkRGAcfPerNsBtZSMDzgehmsI1Sgp261lBBcczEHB+VaEqOdObm+R2zdJUEnqCTPfH0qwX5Uau2/SqdhJbVzBufTacD+epq6TDrUoggjFPx9CZC7SdSkPfic0HQNO6NpG4G469/epoS1bQ8OYWlS4Yc15Jyc5IyeuIGKyls8zlEHRE8yevUH+fpQuF2T6fvaLt9xbZMJHVAz1BnrE743oxYR5lKG3LhhlhDq0r1ayYJ9enx6VlVs2llDjZKVpa5RcBxHqJgwc/CjpfDL3KcUhEuBCZwTiTHc+3r2obN0l25cZ5ToQDoBW0sTETEjbIyJG9WUHmGWw8oLJGVgQD8P2mmLcBsBKJgbUAWwbAASgtiIRMR0x2x86MhGlpDcnyJ0BZwdhuNqsphzGuYIIEHPSsgAgk9e9YBSANasrMJHUmjIgmAJ7mNqIEjy9SR6npUmUpJkA46xvU0EeknbFMiI33q0gWxWA4JQQfbauf/AGjKL9za26HMW3+IUgf1HIH710VaUttnACN8Vxrx9xFq1HEuIoLhuy3ymANgVmEY9z+tBl6oPH2c3ZGAY3o3rQ2wVR19KJ1wa8Yz6lZjTFDInMTRSQJkg9NqGFYiqIegggKHvUfbNYWTzDn2rAJJg0dEAveVHrShTqE03cZBpcqASMZ7U2KAkDSkhUGjKTG4jFLoW5MKo6STvOKZtFgXh5fhVW6nzb7ntVu8CEHFV6hg7d6KAxvgRdAk5xSb0EEU2+CVfGh8uSrSPetMEYM7sQt3Tb39u6P+zIPyNfRHAXAVNqGyxNfOty0U+bsqM13nwU+bng9g71LSZ94rXD6PNapUzfgAUT3obZAUsHqqKmxsD3FQaaLrrpyAHJn4CtLOcOtNj7sdYEAZxMfCjs3ZUg6kEIghRPSCJ29DNZZyIxUlNpCzvO3zo1wLHNWtBAJB9qFwx/ntIlBDqB5hIJR8vjWdcBAcCzMbAnM42/gqseDrPEEBtb50pQ46hKwA9mAZP6Y3HtR2DRavpbK0KWBo1IQCYGZEGe8iMbyPh4sLdWhS2mAhH4iXCSSHYiY6iDvI2GOwHlchHMbRrhsxbNxJWIEIBMDp6DOczVlbQ2hBkrWsa995O/1o0CFCvJ3HesIISNRSQCcwJ9KL5S3qTBj9aWQtxTjam8taTIAySYgyegz/AAVYIcDmOBxtwLgkHOPb50ZJIMHY1hA0rOkAg9evtRUgqg/rUKPIKSYKtxtMGmHWw42UkkT/ANJg0s7bcyQsSjSQQc70dBLZAJB7AnNEmCyv8QXH3ewWASFueQfz2muDfaVfpfv7bhzGzQ+8uD46ED/+z8BXX/F10PvyEgyG05HYnP6RXzlfcQ+/8SvOIiYvbiW4/wDLR5Gx9Cv40nI7YyC4HWk4Pzo2IPQdRXkQP/dvvXtQBOxryB9PAOtxsc+1Q5RG1GXncTivJkjYD1mrIB0wcjasGDtE0RaCRjahKSRkxVogN6OWrNIHyrBg4+lPKJmDtQMFcH4dKfjYvIgCRI83ejoEfGvQAYwBU1ON7AzP8ijYEWroC6JbIqsI3HWrYyUSAYPWq27GkGOtXAKT4EFZIJyD61lKAJCd6mhvuCJ3onLgVoRhyclPeAlKmx1INdg+zB4L8PWvXQVIn2JrlTzXMVddwJ+VdH+yd4O8JfanLb36gVrx9HA1vZ1lnDII3GaPbrOsJjc/TP8ApS9nJaFMWQkAnPUVqRymWCGtOR1qbohQcSBJESrYfD+bVJog5mmIQsRuOtNoSBtkFoBKys6calRK/XGP0qNzbOPpcbLiEczDe+URmYj1z0nvWQjWVtwQYI3z2mfgKOgcy15bejWBgdP9qIopLNm9/wCJId0obPJLRamUSIIIEzo1kiOwjrFXHAHfJyCZQJLZLZbJHU6Dnc7kZ+UtKZ51o5olBWB/T6T2rCGy5e2ygUBAStYEQZwPTodiD0+MSojdljAAEYpfT/5YCCCMb4+faaYjBSTMRsK8gZXGc9vp/O9GAFaSDGJ9aIhPWCPQ1JoZxRUdZgDpVlAlTG3vSN5cBhRfcADbSSST2qzjrWueLnEptQ3sHDBIMQBnPpVSdIpdnO/tM4qbXwxdvsK/xd6eUyQrqvqPYT8q5C0yG3A3s20kISO/TH1raPHfEnbzxUxZpSPulo1rI385iAexj9a19TZUDr0EIEkfz1IpFj0h8xJMSTWZASARREcOUTm6bzvGaKqwYQPx7la/RCYxXllBn0lziJrcSAentQVXISiAfQU1cs2zZBQ0tyDPnVE0Bx5LAKkWbc5IKzI9KLaXvXgWN2rAQ2TPpXv8W4JSy4fWNqKvi9wOWEMtoKzEtgA+9MJVcCDz1j0gUWxFb39ISTw+/eB/CCR6miI4M7MvXLaKmty4CzrdmdjERWUSR5wfjTFFIVJyZhPDrJJE3TivQCppYs2iFNtlztNDSkTjFFGDmT6CisH2/LB3TvMb0hsJAqjvhmFVevAacd6pb1EqJ7Yoo8sjSS4FNABkVIpkTXmkSCIxTCG5imiZITKQA4Y3G/atl+yq65Kr9IknQhcdzmqa+ttFrqHcVY/Zs5p4061O7RWPgsVpx9Hn9ZTbaO62f+TFM2ySHkFGARFJ2P8Ak43FWlo2OUARtW2Jx2MKB3Ao9uQtsKkHETNQj50ZpMCMAT7b/wA+tNFkmmyX5g6P1NSQ2earBABx61K2cnEQeo7V5bY+8N3BcWChJRpBwdZG47iPqasEOy2UNhLipE4IxIqLLKRdrukYW4kTneNsfH615o8yUryB+hojadBRp0ADyHG4/k/zBIEMoJWgpcSFoXggiZB/3oqAVKQQZHf0qCeYSCB5IO+80dB0qQSnc7xtVlB2xG1Z2O1ZKQqDAkGRWYyTVgg4JnX3xFaF4nu7ZT95cXUC2t0GSoyiBkn+/tW8cReFtZOukwEIJrg/jLjzfEOBGyYQ4HXn+W7IA8gy4cdOn/5UrI/oOK+zSnrg3T93eLEOXbpfM7gHYfAQPhVYpTjgKUK0BRq0vEgIWoHpgRVa0lx57YaBkmgsYj//2Q=="

In [16]:
im = Image.open(BytesIO(base64.b64decode(BASE_64)))
im.save(ROOT_DIR + SAVE_PATH + "/Resultado.jpg")
print(ROOT_DIR + SAVE_PATH + "/Resultado.jpg")

In [ ]:
path = ROOT_DIR + SAVE_PATH + "/Resultado.jpg"
print(path)
out = model.predict(path)
print(out)

# Prueba con Webcam

In [ ]:
results = model.predict_webcam()
print(results)

## Otros Intentos

In [ ]:
!pip install pyvirtualdisplay
!pip install opencv-python

In [ ]:
from IPython.display import display
from pyvirtualdisplay import Display

from ipywebrtc import CameraStream
from PIL import Image
import CameraStream, ImageRecorder

In [ ]:
display = Display(visible=0, size=(400, 300))
display.start()

In [ ]:
cap = cv2.VideoCapture(0)
while True:
  if frame is not None and not frame.empty():
    ret, frame = cap.read()
    bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Assuming frame is in RGB color space
    cv2.imshow('Frame', bgr_frame)
    #cv2.imshow(frame)
  else:
    print("Empty frame! Check webcam access or frame capturing.")

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0) # says we capture an image from a webcam
if not cap.isOpened():
    print("Error: Unable to access the webcam.")
ret, frame = cap.read()

if not ret:
  frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


pil_im = Image.fromarray(cv2_im)
pil_im.show()

In [ ]:
cam_stream = CameraStream(constraints={'facing_mode': 'user'}, video=True, audio=False)
